## Usage Data Graphs
- Acceptance Rate per filter;
- n manual invocations per filter;
- Acceptance Rate per user over a session; to see whether there are temporal changes

In [10]:
import os, subprocess, json, pandas as pd, matplotlib.pyplot as plt, datetime as dt
while not os.getcwd().endswith('-analysis'): os.chdir('..')

SERVER_DATA_DIR = 'REDACTED' 
DATA_DIR = os.path.join('..', 'data')

_ = subprocess.run(['rsync', '-avz', f'larisa:{SERVER_DATA_DIR}', DATA_DIR])
DATA_DIR = os.path.join(DATA_DIR, SERVER_DATA_DIR.split('/')[-1])

Authenticated to larisa.ewi.tudelft.nl ([130.161.252.12]:22) using "publickey".


receiving file list ... done
data_aral/0787982c3ee69ffbf34a19d65a92a70f/
data_aral/0787982c3ee69ffbf34a19d65a92a70f/003767253b9442139369be188d8db0c0.json
data_aral/0787982c3ee69ffbf34a19d65a92a70f/0040864f5bad41bfba7a15ee757c1121.json
data_aral/0787982c3ee69ffbf34a19d65a92a70f/01310138a9a34f7685f9bcd28ee194de.json
data_aral/0787982c3ee69ffbf34a19d65a92a70f/017ef7fc38dd47c3be0ec5e44b8f872b.json
data_aral/0787982c3ee69ffbf34a19d65a92a70f/01c6efe66ef041ab96265ce125769185.json
data_aral/0787982c3ee69ffbf34a19d65a92a70f/021cdd7f4598414cb3c7cb29c64c1255.json
data_aral/0787982c3ee69ffbf34a19d65a92a70f/0230bef96e7e4b23a046edc5b6ea0420.json
data_aral/0787982c3ee69ffbf34a19d65a92a70f/0233c9817b504b909b88c617c39b0af0.json
data_aral/0787982c3ee69ffbf34a19d65a92a70f/02bcf63d3bed4ebca613b740784556d7.json
data_aral/0787982c3ee69ffbf34a19d65a92a70f/02c947732aef4d86b115a04282110d84.json
data_aral/0787982c3ee69ffbf34a19d65a92a70f/0314670d13b949719b14ea4a31bea4a9.json
data_aral/0787982c3ee69ffbf34a19d65a

Transferred: sent 165688, received 13722796 bytes, in 2.6 seconds
Bytes per second: sent 63180.5, received 5232803.1


In [11]:
import datetime as dt, json

from pprint import pprint
from dataclasses import dataclass
from typing import Optional

@dataclass
class Query:

    user            : str
    prefix          : str
    suffix          : str
    trigger         : str
    language        : str
    ide             : str
    version         : str
    store           : bool
    timestamp       : dt.datetime
    predictions     : dict
    predict_time    : float
    survey          : bool

    time_since_last_completion: float
    filter_type     : str
    filter_time     : float
    should_filter   : bool
    study_version   : str

    @classmethod 
    def from_dict(cls, data: dict) -> 'Query':
        ''' Parse json dict into pythonic types, returning Query or VerifiedQuery '''

        try: 
            data['timestamp'] = dt.datetime.fromisoformat(data['timestamp'])
            return VerifiedQuery.from_dict(data) \
                if 'verifyToken' in data else cls(**data)
        except Exception as e:
            print(f'Error parsing query: {e}')
            pprint(data)

@dataclass 
class VerifiedQuery(Query):
    verify_token    : str
    chosen_model    : str
    ground_truth    : str
    shown_times     : Optional[list[dt.datetime]]
    accept_time     : dt.datetime

    @classmethod
    def from_dict(cls, data: dict):
        ''' parse json dict into pythonic types '''

        data['verify_token'] = data.pop('verifyToken')
        data['accept_time'] = dt.datetime.fromisoformat(data['accept_time'])
        
        # pushed bugfix for shown_times field on client-side
        data['shown_times'] = [dt.datetime.fromisoformat(t) for t in data['shown_times']] \
            if 'shown_times' in data else None

        return cls(**data)

In [12]:
unfiltered_queries_per_user = {
    user: sorted([
        Query.from_dict({
            **json.load(open(os.path.join(DATA_DIR, user, query))),
            'user': user
        }) for query in os.listdir(os.path.join(DATA_DIR, user))
    ], key=lambda q: q.timestamp, reverse=True) 
    for user in os.listdir(DATA_DIR) if user != '.DS_Store'
}

In [13]:
oops_date = dt.datetime(2024, 3, 21, 17, 17, 50)
queries_per_user = {
    user: [q for q in queries if q.timestamp >= oops_date] \
    for user, queries in unfiltered_queries_per_user.items()
}
oops_queries = [q for q in sum(unfiltered_queries_per_user.values(), []) if q.timestamp < oops_date]
len(oops_queries), len(sum(queries_per_user.values(), []))

(15047, 71592)

#### Recovering Logs
I accidentally forgot to save filtered-out queries in the first 24 hours of the user study; which corresponds to about 9253 data points. Luckily I have a log file, which i pre-process with the following pipeline: 

```sh
cat two_clean.log | grep -v 'POST' | sed 's/-1  | //g' | sed '/^Traceback/,/^RuntimeError/d' | awk '/^\[2024/ {if (NR>1) print ""; print} !/^\[2024/ {printf " %s", $0}' > two_cleaner.log
```

In [14]:
import tqdm 
with open('notebooks/paper/two_cleaner.log', 'r') as f: 
    logs = f.readlines()

# if the line does not start with [2024, append it to the previous line
clean_logs = [] 
for line in logs: 
    if line.startswith('['): 
        clean_logs.append(line)
    else: 
        clean_logs[-1] += line

# remove 'WARNING in api: ' from each line 
clean_logs = [line.replace('WARNING in api: ', '') for line in clean_logs]

# remove any lines that contain 'Error' 
clean_logs = [line for line in clean_logs if 'Error' not in line]

recovered_queries = [] 
fixed_kwargs = dict(user='rec', prefix='', suffix='', trigger='', language='', ide='', version='', store=False, 
                    predictions=dict(), predict_time=0.0, survey=False, time_since_last_completion=0.0,
                    filter_time=0.0, study_version='')

for line in tqdm.tqdm(clean_logs): 
    for filter_type in ['no_filter', 'feature', 'context', 'joint_a', 'joint_h']: 
        if filter_type in line: 

            filter_type = filter_type
            timestamp = dt.datetime.fromisoformat(line[1:24].replace(' ', 'T'))
            should_filter = 'filter' in line[:34]

            query = Query(**fixed_kwargs, timestamp=timestamp, filter_type=filter_type, should_filter=should_filter)
            recovered_queries.append(query)
            break

print(f'Recovered {len(recovered_queries)} queries')

100%|██████████| 9115/9115 [00:00<00:00, 481900.33it/s]

Recovered 9112 queries


In [15]:
# So, recovered_queries contains both the filtered and unfiltered queries. (9'112 from the last log)
# while, oops_queries contains only the unfiltered queries. (14'047 including but also before the last log)
# to maintain the same data distribution, we need to merge them. 

# NOTE: oops queries contains 18 filtered queries (one for each user) which are overriden each time
# let's filter those out first to avoid duplicates. 

unfiltered_queries = [q for q in oops_queries if not q.should_filter]

# NOTE: oops_queries starts much before recovered_queries. 
unfiltered_queries = sorted(unfiltered_queries, key=lambda q: q.timestamp)
recovered_queries = sorted(recovered_queries, key=lambda q: q.timestamp)

print(f'first oops query: {unfiltered_queries[0].timestamp}')
print(f'first recovered query: {recovered_queries[0].timestamp}')

# modify timestamp as the os time on larisa is not set correctly 
filter_timestamp = recovered_queries[0].timestamp + dt.timedelta(minutes=26)

# So, let's remove the oops_queries with timestamp < first recovered query timestamp
unfiltered_queries = [q for q in unfiltered_queries if q.timestamp >= filter_timestamp]

n_oops_filtered = len([q for q in unfiltered_queries if q.should_filter])
n_oops_unfiltered = len([q for q in unfiltered_queries if not q.should_filter])
print(f'Maintained {len(unfiltered_queries)} oops queries ({n_oops_filtered} filtered, {n_oops_unfiltered} unfiltered)')
n_rec_filtered = len([q for q in recovered_queries if q.should_filter])
n_rec_unfiltered = len([q for q in recovered_queries if not q.should_filter])
print(f'Merging with {len(recovered_queries)} recovered queries ({n_rec_filtered} filtered, {n_rec_unfiltered} unfiltered)')

# add the filtered recovered queires to the oops_queries
unfiltered_queries += [q for q in recovered_queries if q.should_filter]

print(f'Final count: {len(unfiltered_queries)} queries') # off by ~4/9000 but good enough

first oops query: 2024-03-17 15:48:44.953718
first recovered query: 2024-03-20 09:43:43.963000
Maintained 6744 oops queries (0 filtered, 6744 unfiltered)
Merging with 9112 recovered queries (2372 filtered, 6740 unfiltered)
Final count: 9116 queries


In [16]:
n_users = set(q.user for q in unfiltered_queries)
print(f'Found {len(n_users)-1} users (excl the recovered one which we cannot match )')

# add unfiltred queries to the queries_per_user dict
for query in unfiltered_queries: 
    if query.user not in queries_per_user: 
        queries_per_user[query.user] = []
    queries_per_user[query.user].append(query)

Found 12 users (excl the recovered one which we cannot match )


In [17]:
# exclude queries where ground-truth is empty 
# exclude queries where the predictions are empty
n_empty = 0
for user, queries in queries_per_user.items(): 
    n_empty += len([q for q in queries if not q.predictions or len(q.predictions) == 0])
print(f'{n_empty} queries have empty predictions')

queries_per_user = {
    user: [q for q in queries if 
        #    q.ground_truth and 
           len(q.predictions) > 0
        ]
    for user, queries in queries_per_user.items()
}

23376 queries have empty predictions


#### Computing Stats

In [18]:
n_queries = sum(len(queries) for queries in queries_per_user.values())
n_users = len(queries_per_user)
n_verified = len(verified := \
                [q for queries in queries_per_user.values() 
                for q in queries if isinstance(q, VerifiedQuery)]
            )
n_filtered = len(filtered := \
                 [q for queries in queries_per_user.values()
                 for q in queries if q.should_filter])

all_queries = sum(queries_per_user.values(), [])
queries_per_filter = {} 
for query in all_queries: 
    filter_type = query.filter_type
    if filter_type not in queries_per_filter: 
        queries_per_filter[filter_type] = [query]
    else: 
        queries_per_filter[filter_type].append(query)

# create a sorted_dict based on the keys 
queries_per_filter = dict(sorted(queries_per_filter.items(), key=lambda x: x[0]))

def tab(list_to_tabulate: list) -> str:
    return "\t".join([f'{i:<15}' if isinstance(i, str) else f'{i:<15.1f}' for i in list_to_tabulate])

def perc(list_1, list_2) -> list:
    return [f'{v:<5} \033[37m({(100*v/n) if n > 0 else 0 :.2f}%)\033[0m' for v, n in zip(list_1, list_2)]

queries_per_filter = dict(sorted(queries_per_filter.items(), key=lambda x: x[0]))

filters = tab(queries_per_filter.keys())
n_per_filter = [len(v) for v in queries_per_filter.values()]
f_per_filter = [len([q for q in v if q.should_filter == True]) for v in queries_per_filter.values()]
i_per_filter = [len([q for q in v if not q.should_filter]) for v in queries_per_filter.values()]
v_per_filter = [len([q for q in v if isinstance(q, VerifiedQuery)]) for v in queries_per_filter.values()]

# NOTE: Filtering out any queries with <10 chars
not_pre_filter_per_filter = [len([q for q in v if (len(q.prefix) + len(q.suffix)) < 10]) for v in queries_per_filter.values()]
pre_filter_per_filter = [len([q for q in v if (len(q.prefix) + len(q.suffix)) >= 10]) for v in queries_per_filter.values()]
passed_per_filter = [len([q for q in v_2 if not q.should_filter or q.trigger == 'manual']) for v_2 in \
                     [[q for q in v if (len(q.prefix) + len(q.suffix)) >= 10] for v in queries_per_filter.values()]]
perceived_v_per_filter = perc([len([q for q in v if isinstance(q, VerifiedQuery)]) for v in queries_per_filter.values()], passed_per_filter)

man_per_filter  = [len([q for q in v if q.trigger == 'manual']) for v in queries_per_filter.values()]
idle_per_filter = [len([q for q in v if q.trigger == 'idle']) for v in queries_per_filter.values()]
auto_per_filter = [len([q for q in v if q.trigger == 'auto']) for v in queries_per_filter.values()]

man_filtered_out = [len([q for q in v if q.trigger == 'manual' and q.should_filter]) for v in queries_per_filter.values()]
idle_filtered_out = [len([q for q in v if q.trigger == 'idle' and q.should_filter]) for v in queries_per_filter.values()]
auto_filtered_out = [len([q for q in v if q.trigger == 'auto' and q.should_filter]) for v in queries_per_filter.values()]

avg_filter_time = [sum([q.filter_time for q in v]) / len(v) for v in queries_per_filter.values()]
med_filter_time = [sorted([q.filter_time for q in v])[len(v)//2] for v in queries_per_filter.values()]
std_filter_time = [pd.Series([q.filter_time for q in v]).std() for v in queries_per_filter.values()]

# compute the 2.5th and 97.5th percentile for the filter time
low_filter_time = [sorted([q.filter_time for q in v])[int(0.05*len(v))] for v in queries_per_filter.values()]
high_filter_time = [sorted([q.filter_time for q in v])[int(0.95*len(v))] for v in queries_per_filter.values()]
 
# get the last query's timestamp per filter 
last_query_per_filter = {k: max(v, key=lambda q: q.timestamp) for k, v in queries_per_filter.items()}
# compute timedelta compared to now
time_since_last_query = {k: dt.datetime.now() - v.timestamp - dt.timedelta(hours=1) \
                         for k, v in last_query_per_filter.items()}
# convert to nice string repr with h:m but no s
time_since_last_query = {k: str(time_since_last_query[k]).split('.')[0] for k in time_since_last_query}
time_per_filter = [time_since_last_query[k] for k in queries_per_filter.keys()]
# remove the seconds from the time_since_last_query
# time_per_filter = [':'.join(t.split(':')[:2]) for t in time_per_filter]
time_per_filter = [f'{t} ago' for t in time_per_filter]

acc_rate_per_filter = [100*v/i if i > 0 else 0 for v, i in zip(v_per_filter, i_per_filter)]
rel_acc_per_filter = [f'{100*acc_rate / acc_rate_per_filter[-1]:.1f}%' for acc_rate in acc_rate_per_filter]

shown_suggestion_lengths = [[sum(len(prediction) for prediction in q.predictions.values()) / len(q.predictions)\
                                for q in queries if q.predictions and len(q.predictions) > 0] \
                                    for queries in queries_per_filter.values()]
med_suggestion_lengths = [sorted(lengths)[len(lengths)//2] for lengths in shown_suggestion_lengths]
avg_suggestion_lengths = [sum(lengths) / len(lengths) for lengths in shown_suggestion_lengths]

acc_suggestion_lengths = [[sum(len(prediction) for prediction in q.predictions.values()) / len(q.predictions)\
                                for q in queries if isinstance(q, VerifiedQuery) and len(q.predictions) > 0] \
                                    for queries in queries_per_filter.values()]
med_acc_suggestion_lengths = [sorted(lengths)[len(lengths)//2] for lengths in acc_suggestion_lengths]
avg_acc_suggestion_lengths = [sum(lengths) / len(lengths) for lengths in acc_suggestion_lengths]

print(f'''
    {n_queries} completions for {n_users} users  ({n_filtered} filtered out in total)
    {n_verified} accepted in total ({100*n_verified/n_queries:.2f}%)

    Filter            | \033[1m{filters} \033[0m
    queries           | {tab(n_per_filter)}
    last query        | {tab(time_per_filter)}

    filtered out      | {tab(perc(f_per_filter, n_per_filter))}
    kept              | {tab(perc(i_per_filter, n_per_filter))}

    avg. filter time  | {tab([f'{t:.2f}' for t in avg_filter_time])}
    med. filter time  | {tab([f'{t:.2f}' for t in med_filter_time])}
    ± std.            | {tab([f'{t:.2f}' for t in std_filter_time])}
    5th perc.         | {tab([f'{t:.2f}' for t in low_filter_time])}
    95th perc.        | {tab([f'{t:.2f}' for t in high_filter_time])}

    accs              | {tab(perc(v_per_filter, n_per_filter))}
    perc. accs        | {tab(perc(v_per_filter, i_per_filter))}
    rel. to no_filter | {tab(rel_acc_per_filter)}

    avg. shown length | {tab(avg_suggestion_lengths)}
    med. shown length | {tab(med_suggestion_lengths)}

    avg. acc. length  | {tab(avg_acc_suggestion_lengths)}
    med. acc. length  | {tab(med_acc_suggestion_lengths)}

    man               | {tab(perc(man_per_filter, n_per_filter))}
    ''')


    57332 completions for 45 users  (10 filtered out in total)
    726 accepted in total (1.27%)

    Filter            | context        	feature        	joint_a        	joint_h        	no_filter       
    queries           | 14056.0        	7804.0         	8652.0         	11683.0        	15137.0        
    last query        | 10:09:11 ago   	3:00:58 ago    	2:03:04 ago    	10:07:13 ago   	1:00:28 ago    

    filtered out      | 8     (0.06%)	0     (0.00%)	1     (0.01%)	1     (0.01%)	0     (0.00%)
    kept              | 14048 (99.94%)	7804  (100.00%)	8651  (99.99%)	11682 (99.99%)	15137 (100.00%)

    avg. filter time  | 32.14          	0.19           	35.31          	35.36          	0.02           
    med. filter time  | 23.64          	0.16           	24.42          	18.56          	0.02           
    ± std.            | 27.86          	0.07           	30.23          	41.42          	0.01           
    5th perc.         | 11.21          	0.13           	11.89          	11.30  

In [19]:
#     < 10chr           | {tab(perc(not_pre_filter_per_filter, n_per_filter))}
#     >= 10chr          | {tab(perc(pre_filter_per_filter, n_per_filter))}
#     passed            | {tab(perc(passed_per_filter, pre_filter_per_filter))}
#     perceived accs    | {tab(perceived_v_per_filter)}
# 
#     auto              | {tab(perc(auto_per_filter, n_per_filter))}
#     idle              | {tab(perc(idle_per_filter, n_per_filter))}
# 
#     n man filtered    | {tab(perc(man_filtered_out, man_per_filter))}
#     n auto filtered   | {tab(perc(auto_filtered_out, auto_per_filter))}
#     n idle filtered   | {tab(perc(idle_filtered_out, idle_per_filter))}

In [58]:
stats = {} 
# print three most recent users, and the number of queries in the last 24 hours 
# for user, queries in sorted({k: v for k, v in queries_per_user.items() if len(v) > 0}, key=lambda x: len(x[1])):
for user, queries in sorted(queries_per_user.items(), key=lambda x: len(x[1]), reverse=True):
                            # key=lambda x: x[1][0].timestamp, reverse=True):
    most_recent = (str(dt.datetime.now() - dt.timedelta(hours=1) - queries[0].timestamp).split('.')[0] )\
          if len(queries) > 0 else None
    user_uuid = user[:4]
    vqs = [q for q in queries if isinstance(q, VerifiedQuery)]
    last_day_qs = [q for q in queries if q.timestamp > dt.datetime.now() - dt.timedelta(days=1)]
    last_day_vqs = [q for q in last_day_qs if isinstance(q, VerifiedQuery)]
    last_day_acc_rate = len(last_day_vqs) / len(last_day_qs) if last_day_qs else 0
    last_day_top_three_langs = sorted(
        {q.language for q in queries if q.timestamp > dt.datetime.now() - dt.timedelta(days=1)},
        key=lambda lang: len([q for q in queries if q.language == lang]),
        reverse=True
    )[:3]
    top_langs = sorted(
        {q.language for q in queries},
        key=lambda lang: len([q for q in queries if q.language == lang]),
        reverse = True
    )

    stats[user_uuid] = {
        ('','most recent'): most_recent,
        ('total', 'n'): len(queries),
        ('total', 'acc'): len(vqs),
        ('total', 'acc_rate'): (100*len(vqs) / len(queries)) if len(queries) > 0 else 0,
        ('total', 'top langs'): top_langs,
        ('last day', 'n'): len(last_day_qs),
        ('last day', 'acc'): len(last_day_vqs),
        ('last day', 'rate'): 100*last_day_acc_rate,
        ('last day', 'top langs'): last_day_top_three_langs
    }

In [59]:
# convert tuples to layered headings in dataframe
df = pd.DataFrame(stats).T
df.columns = pd.MultiIndex.from_tuples(df.columns)
df.sort_values(('', 'most recent'), ascending=False)

# print n users with >50 queries 
n_with_50 = len([q for q in queries_per_user.values() if len(q) >= 50])
print(f'\t{n_with_50} users with 50 or more queries')

df

	34 users with 50 or more queries


total                 \
            most recent      n  acc  acc_rate   
1ae4    3 days, 8:11:53  13876   23  0.165754   
8cc7    3 days, 8:56:56  12736   51   0.40044   
a926            6:31:38   7706   88  1.141967   
ed93   2 days, 15:09:49   6252    6  0.095969   
5747           18:11:36   5410   87  1.608133   
f337            6:24:25   4342   33  0.760018   
7f5d            7:02:05   3345  129  3.856502   
0787           21:56:16   2540   15  0.590551   
a420            6:26:30   2419   28  1.157503   
rec    15 days, 8:49:18   2372    0       0.0   
cc39    5 days, 7:21:11   2165   49  2.263279   
f47e    3 days, 1:35:25   2013   11  0.546448   
6064   12 days, 3:03:23   1145    7  0.611354   
2f6a    8 days, 5:06:00   1048   16  1.526718   
c9b7    8 days, 8:07:27    841   10  1.189061   
04db   3 days, 20:33:08    816   11  1.348039   
c715   12 days, 1:37:06    711    3  0.421941   
0178   5 days, 13:30:42    585    9  1.538462   
250b     1 day, 2:02:22    570    5  0.877193   
7cf7   4 days, 18:08:36    401    7  1.745636   
df7d           23:53:21    346    4  1.156069   
8c9f    9 days, 7:47:55    314    2  0.636943   
7dd3   11 days, 3:51:48    308    7  2.272727   
62a9    3 days, 4:33:14    301    9  2.990033   
e575   6 days, 21:09:05    268    1  0.373134   
8caf            6:24:35    209    2  0.956938   
6832     1 day, 3:30:30    189    1  0.529101   
ebdc   12 days, 2:44:24    145    6  4.137931   
53ed            9:06:29    134    2  1.492537   
1c67    4 days, 5:58:58    123    2  1.626016   
cff8    1 day, 16:55:28     70    0       0.0   
4849   7 days, 10:06:33     70    0       0.0   
49ee    1 day, 23:12:26     56    2  3.571429   
c961  13 days, 11:26:00     53    2  3.773585   
90b0    5 days, 8:57:07     46    0       0.0   
a57c   8 days, 11:48:39     23    0       0.0   
b509   5 days, 23:33:15     21    0       0.0   
8166   13 days, 8:48:09      9    0       0.0   
f1d4   3 days, 19:54:51      3    0       0.0   
cbe6   14 days, 0:09:31      2    0       0.0   
5d8f  11 days, 17:57:54      2    0       0.0   
7869    6 days, 3:03:20      1    0       0.0   
cbb6               None      0    0         0   
3a63               None      0    0         0   
4217               None      0    0         0   

                                                        last day      \
                                              top langs        n acc   
1ae4                          [latex, scminput, python]        0   0   
8cc7  [python, latex, markdown, typescript, plaintex...        0   0   
a926  [python, yaml, ignore, latex, jsonc, propertie...      401   5   
ed93                          [latex, python, scminput]        0   0   
5747  [python, typescriptreact, typescript, plaintex...      644   1   
f337  [python, http, markdown, csv, dynamic csv, pip...      382   2   
7f5d    [php, html, css, cpp, python, pip-requirements]        2   0   
0787  [typescript, lua, scss, typescriptreact, prope...      120   1   
a420      [python, markdown, java, search-result, json]      350   3   
rec                                                  []        0   0   
cc39                                 [html, javascript]        0   0   
f47e  [css, html, python, javascript, markdown, json...        0   0   
6064                                 [python, markdown]        0   0   
2f6a                       [markdown, python, cpp, ini]        0   0   
c9b7                       [python, markdown, scminput]        0   0   
04db  [javascript, json, scminput, markdown, html, p...        0   0   
c715           [typescriptreact, typescript, css, json]        0   0   
0178  [typescriptreact, yaml, json, typescript, pyth...        0   0   
250b            [python, jsonc, shellscript, yaml, log]        0   0   
7cf7                           [python, arduino-output]        0   0   
df7d                     [python, scminput, yaml, json]        0   0   
8c9f                                           [python]        0 